In [1]:
# 프로젝트 루트를 Python 경로에 추가
import sys
import os
from pathlib import Path

# 현재 노트북의 위치에서 프로젝트 루트 찾기
current_dir = Path.cwd()
if 'examples' in str(current_dir):
    # examples 폴더에서 실행하는 경우
    project_root = current_dir.parent.parent
else:
    # 프로젝트 루트에서 실행하는 경우
    project_root = current_dir

# 프로젝트 루트를 Python 경로에 추가
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# 필요한 모듈 가져오기
from typing import List, Dict, Any, Optional
import polars as pl
from datetime import datetime

from quantbt import (
    SimpleBacktestEngine, 
    SimpleBroker, 
    TradingStrategy, 
    MarketDataBatch, 
    BacktestConfig,
    UpbitDataProvider,
    Order,
    OrderSide,
    OrderType,
)
print("✅ 모든 QuantBT 모듈이 성공적으로 가져와졌습니다!")

✅ 모든 QuantBT 모듈이 성공적으로 가져와졌습니다!


In [2]:
class SimpleSMAStrategy(TradingStrategy):
    """간단한 SMA 브레이크아웃 전략
    
    매수: 가격이 SMA20 상회
    매도: 가격이 SMA5 하회  
    """
    
    def __init__(self, buy_sma: int = 15, sell_sma: int = 30):
        super().__init__(
            name="SimpleSMAStrategy",
            config={
                "buy_sma": buy_sma,
                "sell_sma": sell_sma
            },
            position_size_pct=0.8,  # 80%씩 포지션
            max_positions=1
        )
        self.buy_sma = buy_sma
        self.sell_sma = sell_sma
        self.indicator_columns = [f"sma_{buy_sma}", f"sma_{sell_sma}"]
        
    def _compute_indicators_for_symbol(self, symbol_data):
        """심볼별 이동평균 지표 계산"""
        
        # 시간순 정렬 확인
        data = symbol_data.sort("timestamp")
        
        # 단순 이동평균 계산
        buy_sma = self.calculate_sma(data["close"], self.buy_sma)
        sell_sma = self.calculate_sma(data["close"], self.sell_sma)
        
        # 지표 컬럼 추가
        return data.with_columns([
            buy_sma.alias(f"sma_{self.buy_sma}"),
            sell_sma.alias(f"sma_{self.sell_sma}")
        ])
    
    def generate_signals(self, data: MarketDataBatch) -> List[Order]:
        """신호 생성 - 가격과 이동평균 비교"""
        orders = []
        
        if not self.context:
            return orders
        
        for symbol in data.symbols:
            current_price = self.get_current_price(symbol, data)
            if not current_price:
                continue
            
            # 현재 지표 값 조회
            buy_sma = self.get_indicator_value(symbol, f"sma_{self.buy_sma}", data)
            sell_sma = self.get_indicator_value(symbol, f"sma_{self.sell_sma}", data)
            
            if buy_sma is None or sell_sma is None:
                continue
            
            current_positions = self.get_current_positions()
            
            # 매수 신호: 가격이 SMA20 상회 + 포지션 없음
            if current_price > buy_sma and symbol not in current_positions:
                portfolio_value = self.get_portfolio_value()
                quantity = self.calculate_position_size(symbol, current_price, portfolio_value)
                
                if quantity > 0:
                    order = Order(
                        symbol=symbol,
                        side=OrderSide.BUY,
                        quantity=quantity,
                        order_type=OrderType.MARKET
                    )
                    orders.append(order)
            
            # 매도 신호: 가격이 SMA5 하회 + 포지션 있음
            elif current_price < sell_sma and symbol in current_positions and current_positions[symbol] > 0:
                order = Order(
                    symbol=symbol,
                    side=OrderSide.SELL,
                    quantity=current_positions[symbol],
                    order_type=OrderType.MARKET
                )
                orders.append(order)
        
        return orders

In [3]:
# 1. 업비트 데이터 프로바이더
upbit_provider = UpbitDataProvider()

In [4]:
# 2. 백테스팅 설정 (2024년 1년)
config = BacktestConfig(
    symbols=["KRW-BTC"],
    start_date=datetime(2024, 1, 1),
    end_date=datetime(2024, 12, 31),
    timeframe="1d",  # 일봉
    initial_cash=10_000_000,  # 1천만원
    commission_rate=0.0,      # 수수료 0% (테스트용) - 실제 백테스팅에는 적절한 값 사용
    slippage_rate=0.0,         # 슬리피지 0% (테스트용) - 실제 백테스팅에는 적절한 값 사용
    save_portfolio_history=True
)

In [5]:
# 3. 간단한 SMA 전략
strategy = SimpleSMAStrategy(
    buy_sma=15,   # 매수: 가격이 20일 이평선 상회
    sell_sma=30    # 매도: 가격이 5일 이평선 하회
)

In [6]:
# 4. 브로커 설정
broker = SimpleBroker(
    initial_cash=config.initial_cash,
    commission_rate=config.commission_rate,
    slippage_rate=config.slippage_rate
)

In [7]:
# 5. 백테스트 엔진
engine = SimpleBacktestEngine()
engine.set_strategy(strategy)
engine.set_data_provider(upbit_provider)
engine.set_broker(broker)

In [8]:
# 6. 백테스팅 실행
print(f"📅 기간: {config.start_date.date()} ~ {config.end_date.date()}")
print(f"📈 전략: 가격 > SMA{strategy.buy_sma} 매수, 가격 < SMA{strategy.sell_sma} 매도")
print(f"💰 초기 자본: {config.initial_cash:,.0f}원")
print(f"📊 수수료: {config.commission_rate:.1%} | 슬리피지: {config.slippage_rate:.1%}")

result = await engine.run(config)

📅 기간: 2024-01-01 ~ 2024-12-31
📈 전략: 가격 > SMA15 매수, 가격 < SMA30 매도
💰 초기 자본: 10,000,000원
📊 수수료: 0.0% | 슬리피지: 0.0%


In [9]:
# 7. 결과 출력
result.print_summary()

                 BACKTEST RESULTS SUMMARY
Period          : 2024-01-01 ~ 2024-12-31
Initial Capital : $10,000,000
Final Equity    : $23,533,560
Total Return    : 135.34%
Annual Return   : 135.47%
Volatility      : 30.53%
Sharpe Ratio    : 4.44
Calmar Ratio    : 11.67
Sortino Ratio   : 6.49
Max Drawdown    : 11.61%
Total Trades    : 24
Win Rate        : 50.0%
Profit Factor   : 13.39
Execution Time  : 0.30s


In [10]:
   
# 시각화 데이터 확인
print("\n주피터 노트북에서 사용 가능한 시각화 메서드:")
print("1. result.plot_portfolio_performance() - 포트폴리오 성과 차트")
print("2. result.plot_returns_distribution() - 수익률 분포 히스토그램")
print("3. result.plot_monthly_returns_heatmap() - 월별 수익률 히트맵")
print("4. result.show_performance_comparison() - 벤치마크 비교 표")

# 샘플 호출 (주피터 노트북 환경이 아니므로 실제 차트는 생성되지 않음)
try:
    print("\n시각화 메서드 호출 테스트...")
    result.plot_portfolio_performance()
    result.plot_returns_distribution(period="daily")
    result.plot_monthly_returns_heatmap()
    result.show_performance_comparison()
except Exception as e:
    print(f"시각화 테스트 중 예상된 오류 (plotly 없음): {e}")




주피터 노트북에서 사용 가능한 시각화 메서드:
1. result.plot_portfolio_performance() - 포트폴리오 성과 차트
2. result.plot_returns_distribution() - 수익률 분포 히스토그램
3. result.plot_monthly_returns_heatmap() - 월별 수익률 히트맵
4. result.show_performance_comparison() - 벤치마크 비교 표

시각화 메서드 호출 테스트...


,Metric,Strategy,Benchmark
0,Total Return (%),135.34,134.35
1,Annual Return (%),135.47,134.49
2,Volatility (%),30.53,46.49
3,Sharpe Ratio,4.44,2.89
4,Calmar Ratio,11.67,4.59
5,Sortino Ratio,6.49,5.06
6,Max Drawdown (%),11.61,29.28
7,Beta,0.52,1.00
8,Alpha,0.66,0.00
9,Total Trades,24,-
